In [ ]:
# # access.log
               #svc#status#ms
# 1st iter:01Z auth   200 120 #total[svc]=120
# 2iter: 02Z   auth   503 80  #total=200                  errors=1
# 3iter: 03Z   users   200 200 #total=200

# # comment
# 4iter: 04Z   users   500 220 #total=420                          errors=2
# 5iter: 05Z   users   404 90 #total=290
# 6iter: 06Z   orders   502 300 #total=300                          errors=3
# 7iter: 07Z   orders   200 310 #total=610
# 8iter: 08Z   orders   200 290 #total=900*
# 9iter: 09Z   auth   200 110  #total=310
# 10iter: 10Z  auth   500 130  #total=440*                           errors=4
# 11iter: 11Z  users   200 210 #total=410
# 12iter: 12Z  users   503 230 #total=640*                       errors=5*


#counts = {"auth":4, "users":5, "orders":3}
#totals = {"auth":440, "users":640, "orders":900}


# errors_total=6
# top_slowest:
# - orders avg_ms=300.0
# - users avg_ms=190.0


import sys

def analyze(lines):
    totals = {}   # service -> total latency
    counts = {}   # service -> count
    errors = 0

    for line in lines:
        line = line.strip()
        if not line or line.startswith("#"): #changed to or
            continue

        parts = line.split() 
        #--> ["ts","users","200","120"] --> list of "str"
        #--> ["ts","users","500","200"] --> list of "str"
        #2026-02-07T10:00:02Z auth 503 80
        #2026-02-07T10:00:03Z users 200 200

        svc = parts[1] 
        # svc = "auth"-->str
        #auth
        # svc = "users"-->str


        status = int(parts[2])  
        # status = 200
        #503
        # status = 500

        ms = int(parts[3])
        # ms = 120
        #80
        # ms = 220

        totals[svc] = totals.get(svc,0) + ms
        # {"auth":120}
        # after 2nd iter: {"auth":200}

        # user's status = "500"

        counts[svc] = counts.get(svc, 0) + 1
        # {"auth":2}

        # {"users":1}

        if status >= 500:
            errors += 1
            #error = 1
            #error = 
    #totals = {"auth":440, "users":640, "orders":900}
    #counts = {"auth":4, "users":5, "orders":3}
    avgs = {}
    for svc in totals:
        avgs[svc] = totals[svc] / counts[svc] #(pp1 if counts is a None - lets comeback to this later)

    
    top2 = sorted(avgs, key = lambda kv:kv[1], reverse=True)[:2] 
    return errors, top2, avgs
    #top2 = {"orders":180, "users":128}
    #avg = {"auth":110, "users":128, "orders":300}
    #errors = 5


def main(argv):
    if len(argv) != 1:
        raise ValueError("usage: slow.py <logfile>")

    with open(argv[0]) as f:
        errors, top2, avgs = analyze(f)
    #top2 = {"orders":180, "users":128}
    #avg = {"auth":110, "users":128, "orders":180}
    #errors = 5

    print("errors_total=" + str(errors)) #"errors_total=5"
    print("top_slowest:") #top_slowest:
                            #- orders avg_ms=300
                            #- users avg_ms=128
    for svc in top2:
        print("- " + svc + " avg_ms=" + str(avgs[svc]))

if __name__ == "__main__":
    main(sys.argv[1:])